In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import datetime
import time
import os

In [3]:
# ----------- Step 1: Scrape latest BTCL price from RSEBL -----------
def get_latest_btcl_price():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    try:
        url = "https://www.rsebl.org.bt/"
        driver.get(url)
        time.sleep(5)

        rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            if cols and "BTCL" in cols[0].text:
                return float(cols[2].text.strip())
    finally:
        driver.quit()

In [5]:
# ----------- Step 2: Prepare data and features -----------
def prepare_data(df, window_size=60):
    prices = df['BTCL'].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    scaled_prices = scaler.fit_transform(prices)

    X, y = [], []
    for i in range(window_size, len(scaled_prices)):
        X.append(scaled_prices[i - window_size:i, 0])
        y.append(scaled_prices[i, 0])
    X = np.array(X).reshape(-1, window_size, 1)
    y = np.array(y)

    split = int(len(X) * 0.8)
    return X[:split], X[split:], y[:split], y[split:], scaler, scaled_prices

In [7]:
# ----------- Step 3: Train LSTM model -----------
def train_model(X_train, y_train, X_test, y_test):
    model = Sequential([
        Input(shape=(X_train.shape[1], 1)),
        LSTM(100, return_sequences=True),
        LSTM(100),
        Dense(1)
    ])
    model.compile(optimizer=Adam(), loss='mean_squared_error')
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stop])
    return model

In [8]:
# ----------- Step 4: Predict next-day price -----------
def predict_next_day(model, scaled_prices, latest_price, scaler, window_size=60):
    latest_scaled = scaler.transform(np.array([[latest_price]]))
    updated_scaled = np.append(scaled_prices, latest_scaled, axis=0)
    sequence = updated_scaled[-window_size:].reshape(1, window_size, 1)
    next_day_scaled = model.predict(sequence, verbose=0)
    predicted_price = scaler.inverse_transform(next_day_scaled)
    return predicted_price[0][0]


In [11]:
# ----------- Step 5: Main pipeline -----------
def main():
    # Load your historical BTCL stock data
    csv_path = "/Users/ugyen/Documents/GitHub/Stockex-btn/Dataset/BTCL.csv"
    df = pd.read_csv(csv_path)
    df['DateTime'] = pd.to_datetime(df['DateTime']).dt.date
    df = df.groupby('DateTime').last().reset_index()

    # Prepare training data
    window_size = 60
    X_train, X_test, y_train, y_test, scaler, scaled_prices = prepare_data(df, window_size)

    # Train model
    model = train_model(X_train, y_train, X_test, y_test)

    # Scrape latest price
    latest_price = get_latest_btcl_price()
    if latest_price is None:
        print("Failed to retrieve latest price from RSEBL website.")
        return

    print(f"Latest BTCL price scraped: Nu. {latest_price}")

    # Predict next-day price
    predicted_price = predict_next_day(model, scaled_prices, latest_price, scaler, window_size)
    print(f"Predicted next-day price based on latest: Nu. {predicted_price:.2f}")

    # Optional: Append scraped price to CSV
    today = datetime.date.today()
    df = pd.concat([df, pd.DataFrame({'DateTime': [today], 'BTCL': [latest_price]})], ignore_index=True)
    df.to_csv("/Users/ugyen/Documents/GitHub/Stockex-btn/updated_stock_data.csv", index=False)

# Run
if __name__ == "__main__":
    main()


Epoch 1/50


KeyboardInterrupt: 